# HW 3 Convex Optimisation

Quelques fonctions pour faciliter les résolutions :

In [25]:
## Shape des variables

# A --> R^n*2d
# Q --> R^n**2
# v, p --> R^n
# b --> R^2n

###

import numpy as np
import cvxpy as cp



## Question 2 :

a) Resolution avec Newton :

In [55]:
import numpy as np

def centering_step(Q, p, b, A, t, v0, eps, max_iter):
    """
    Arguments:
    Q -- Matrice définissant le terme quadratique dans l'objectif.
    p -- Vecteur définissant le terme linéaire dans l'objectif.
    b -- Vecteur des bornes pour les contraintes.
    A -- Matrice des coefficients des contraintes.
    t -- Paramètre d'échelle de la barrière logarithmique.
    v0 -- Point de départ pour l'algorithme.
    eps -- Tolérance pour le critère d'arrêt.
    max_iter -- Nombre maximal d'itérations autorisé.

    Retourne:
    v -- Solution optimale trouvée.
    """

    def f(v, Q, p, b, A, t):
        term1 = v.T @ Q @ v + p.T @ v
        term2 = np.sum(np.log(b - A.T @ v))
        return t * term1 - term2

    def grad_f(v, Q, p, b, A, t):
        return t * (2 * Q @ v + p) + ((1 / (b - A.T @ v)).T @ A).T

    def hess_f(v, Q, b, A, t):
        diag_vals = 1 / (b - A.T @ v)**2
        return 2 * t * Q + A @ np.diag(diag_vals) @ A.T

    def newton_step(hess, grad):
        return -np.linalg.solve(hess, grad)

    def decrement(hess, grad):
        return grad.T @ np.linalg.solve(hess, grad)

    def backtracking_linesearch(v, delta_v):
        beta = 0.5
        alpha = 1e-4
        t_step = 1.0
        while f(v + t_step * delta_v, Q, p, b, A, t) > f(v, Q, p, b, A, t) + alpha * t_step * grad_f(v, Q, p, b, A, t).T @ delta_v:
            t_step *= beta
        return t_step

    # Initialisation
    v = v0
    iter_count = 0

    while iter_count < max_iter:
        grad = grad_f(v, Q, p, b, A, t)
        hess = hess_f(v, Q, b, A, t)

        delta_v = newton_step(hess, grad)
        lambda_2 = decrement(hess, grad)

        if lambda_2 / 2 < eps:
            break

        step_size = backtracking_linesearch(v, delta_v)
        v = v + step_size * delta_v
        iter_count += 1

    return v


In [60]:
def barr_method(Q, p, A, b, v0, eps, mu=10, max_iter=100):
    """
    Méthode de barrière pour résoudre un problème d'optimisation quadratique avec contraintes.

    Arguments:
    Q -- Matrice définissant le terme quadratique dans l'objectif.
    p -- Vecteur définissant le terme linéaire dans l'objectif.
    A -- Matrice des coefficients des contraintes.
    b -- Vecteur des bornes pour les contraintes.
    v0 -- Point de départ pour l'algorithme.
    eps -- Tolérance pour le critère d'arrêt.
    mu -- Facteur d'augmentation du paramètre t.
    max_iter -- Nombre maximal d'itérations autorisé.

    Retourne:
    v_seq -- Séquence des solutions trouvées à chaque itération.
    prec_criterion -- Liste des critères de précision calculés.
    """

    # Initialisation du paramètre t
    t = 1

    # Historique des solutions et du critère de précision
    v_seq = []
    prec_criterion = []

    for _ in range(max_iter):
        # 1. Étape de centrage en utilisant la méthode de Newton
        v = centering_step(Q, p, b, A, t, v0, eps, max_iter)
        v_seq.append(v)
        
        # 2. Mise à jour du point de départ
        v0 = v

        # 3. Calcul du critère de précision
        current_prec = A.shape[0] / t
        prec_criterion.append(current_prec)

        # 4. Critère d'arrêt
        if current_prec < eps:
            break

        # 5. Augmenter t pour rendre la barrière plus contraignante
        t *= mu

    return v_seq, prec_criterion


## Question 3 :

In [61]:
import numpy as np
import matplotlib.pyplot as plt

# Générer des données aléatoires pour X et y
n = 40
d = 50
np.random.seed(0)  # Fixer la graine pour des résultats reproductibles
X = np.random.rand(n, d)
y = np.random.rand(n)

# Fixer epsilon
eps = 1e-8

# Fixer les variables selon la question 1
A = np.concatenate([X, -X], axis=1).T
Q = np.eye(n) * 0.5
p = -y
b = 10 * np.ones(2 * d)  # Lambda = 10

# Variable initiale
v0 = np.zeros(n)

# Différentes valeurs de mu
mus = [2, 15, 50, 100, 250, 500, 1000]

# Listes pour stocker les résultats
prec_criterion = []
gap = []

for mu in mus:
    # Résolution avec la méthode de barrière
    v_seq, prec = barr_method(Q, p, A, b, v0, eps, mu=mu)
    prec_criterion.append(prec)
    
    # Calculer le gap
    v = [v_it for v_it in v_seq]  # Extraire les solutions intermédiaires
    v_opt = v_seq[-1]  # Solution optimale après la dernière itération
    g_opt = v_opt.T @ Q @ v_opt + p.T @ v_opt  # Calculer le coût optimal

    # Calcul des gaps pour toutes les solutions intermédiaires
    gap.append([v_i.T @ Q @ v_i + p.T @ v_i - g_opt for v_i in v[:-1]])

# Tracer les résultats
plt.figure(figsize=(12, 6))

# Précision en fonction de mu
for i, prec in enumerate(prec_criterion):
    plt.plot(range(len(prec)), prec, label=f'mu = {mus[i]}')
plt.xlabel('Iterations')
plt.ylabel('Precision Criterion')
plt.title('Precision Criterion vs Iterations for Different mu')
plt.legend()
plt.show()

# Gap en fonction de mu
plt.figure(figsize=(12, 6))
for i, g in enumerate(gap):
    plt.plot(range(len(g)), g, label=f'mu = {mus[i]}')
plt.xlabel('Iterations')
plt.ylabel('Gap')
plt.title('Gap vs Iterations for Different mu')
plt.legend()
plt.show()


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 40 is different from 100)

In [43]:
v = centering_step(Q, p, b, A, t, f, grad_f, hess_f, v0, 1e-6,1000)

In [32]:
barrier_method(Q, p, b, A, v0, eps=1e-6, mu=30)

Iteration: 1, t: 0.1


TypeError: centering_step() missing 4 required positional arguments: 'hess_f', 'v0', 'eps', and 'max_iter'

In [37]:
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize((cp.quad_form(x, Q) + p.T @ x)), [A @ x <= b])
print()
print("------------------------------------------------------------------")
v_sol = v
fx_sol = prob.solve()
fv_sol = np.dot(v_sol, Q@v_sol) + np.dot(p.T, v_sol)
print("Difference between min values: ", np.abs(fx_sol - fv_sol))
print()
print("v_sol (my solution):", v_sol)
print("x_sol (cvxpy solution):", x.value)


------------------------------------------------------------------
Difference between min values:  2.292880759721299

v_sol (my solution): [ 5.80976613e-16  4.72664339e-16  6.22477313e-16  7.92748553e-16
  4.45539017e-16  7.52815869e-16 -3.36307752e-16 -1.67657413e-16
 -3.06113700e-16 -9.61701537e-17]
x_sol (cvxpy solution): [0.96958463 0.77513282 0.93949894 0.89482735 0.59789998 0.92187424
 0.0884925  0.19598286 0.04522729 0.32533033]
